<a href="https://colab.research.google.com/github/Ederson-Branco/Data_Science/blob/main/pdf_to_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala a biblioteca tabula-py para leitura do pdf
!pip install tabula-py

     |████████████████████████████████| 11.7MB 5.1MB/s 


In [ ]:
'''
Importando as bibliotecas e carregando o documento pdf
'''

from tabula import read_pdf
import pandas as pd
df2 = read_pdf('/content/conector.pdf',pages="all",guess=False)
df3 = pd.DataFrame(df2)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [ ]:
'''
Função usada para excluir linhas vazias que são geradas quando existe quebra de 
linha no nome da norma
'''
def filtra_linhas_nao_vazias(serie,index):
  coluna_2 = df3[0][index].columns[2]
  selecao = df3[0][index][coluna_2].notna()
  return serie[selecao]

In [ ]:
'''
Função para extrair o nome do colaborador
'''
def retorna_nome(index):

  coluna_2 = df3[0][index].columns[2]

  # Extraindo a linha 4 da coluna 2, onde está o nome
  nome_agrupado = df3[0][index][coluna_2].iloc[4]
  nome_agrupado = nome_agrupado.split()

  # Laço que seleciona o nome (Quando possui só letras maiúscula) 
  nome_colaborador_lista = []
  for i in nome_agrupado:
    if i == i.upper():
      nome_colaborador_lista.append(i)

  # Laço para alterar o nome de lista para string
  nome_colaborador_string = ''
  for i in nome_colaborador_lista:
      nome_colaborador_string = nome_colaborador_string + ' ' + i
  nome_colaborador_string = nome_colaborador_string.strip() #Exclui o primeiro espaço em branco

  return nome_colaborador_string

In [ ]:
'''
Função para extrair a listagem de normas e o código de conhecimento
'''

def retorna_lista_normas(index):

  # Slecionando a coluna 1 onde estão os dados das normas
  coluna_1 = df3[0][index].columns[0]
  lista_normas = df3[0][index][coluna_1]
  lista_normas = filtra_linhas_nao_vazias(lista_normas,index)

  conhecimento = []
  normas = []
  # Laço para extrair o código do conhecimento e o nome da norma
  for i,j in enumerate(lista_normas):
    if i > 4 and i < len(lista_normas) - 1:
      normas_split = j.split()
      conhecimento.append(normas_split[0])

      normas_string = ''
      # Laço para alterar o nome da norma de lista para string
      for i,j in enumerate(normas_split):
        if i > 0 :
          normas_string = normas_string + ' ' + j
      normas.append(normas_string.strip()) #Exclui o primeiro espaço em branco
      
  return conhecimento,normas

In [ ]:
'''
Função para extrair o número de revisão da norma e da tlt
'''

def retorna_revisoes(index):
  coluna_2 = df3[0][index].columns[2]
  lista_revisoes = df3[0][index][coluna_2]
  lista_revisoes = filtra_linhas_nao_vazias(lista_revisoes,index)
  revisao_norma = []
  revisao_tlt = []

  # Laço para extrair o número da revisão_tlt e revisão_norma
  for i,j in enumerate(lista_revisoes):
    if i > 4 and i < len(lista_revisoes) - 1:
      revisao_norma.append(j.split()[-1])
      # Coloca um zero quando o valor está em branco
      if j.split()[-3] ==  'a':
        revisao_tlt.append('0')
      else:
        revisao_tlt.append((j.split()[-3]))

  return revisao_tlt,revisao_norma


In [ ]:
'''
Criando o dataframe
'''
dataframe = pd.DataFrame()                                                
for index in range(df3.shape[0]):  
  colaborador = retorna_nome(index)                                      
  conhecimento,descricao = retorna_lista_normas(index)
  rev_tlt,rev_normas = retorna_revisoes(index)

  dados = {'Conhecimento':conhecimento,'Descrição_Norma':descricao,'Colaborador': colaborador,
            'Rev_TLT':rev_tlt,'Rev_Norma':rev_normas}

  dataframe = pd.concat([dataframe,pd.DataFrame(dados)],ignore_index=True) 

dataframe

,Conhecimento,Descrição_Norma,Colaborador,Rev_TLT,Rev_Norma
0,31252,WPM-1453 PT : NÃO CONFORMIDADES E AÇÕES CORRET...,ADEMIR DOS SANTOS,11,18
1,31394,WPM-2094 PT : ADMINISTRAR GARANTIAS DE PRODUTO,ADEMIR DOS SANTOS,21,24
2,31742,WPM-2350 PT : PROCESSAR DEVOLUÇÕES DE CLIENTES,ADEMIR DOS SANTOS,6,11
3,30429,WPM-24063 PT : CONTROLE DE DOCUMENTOS DEPARTAM...,ADEMIR DOS SANTOS,2,4
4,31501,WPR-24891 PT : TRATATIVA PARA NÃO CONFORMIDADE...,ADEMIR DOS SANTOS,1,3
...,...,...,...,...,...
215,26628,WPS-4459 PT : DETERMINAÇÃO DA COR DE PIGMENTOS...,VANDERLEI BERTOL,2,3
216,22474,WPS-6301 PT : PLANO DE AMOSTRAGEM PARA MATÉRIA...,VANDERLEI BERTOL,21,23
217,40322,WPS-27244 PT : PROCEDIMENTOS PARA DISSIPAÇÃO D...,VANESSA SCHMIDT,1,2
218,41855,WFR-27522 PT : KaizenWEG - Sugestão,WILLIAN SILVA DE SOUSA,3,4


In [ ]:
'''
Convertendo valores numéricos em int64
'''
dataframe[['Conhecimento','Rev_TLT','Rev_Norma']] = dataframe[['Conhecimento','Rev_TLT','Rev_Norma']].astype('int')

In [ ]:
'''
Verificando a listagem de colaboradores
'''
dataframe.Colaborador.unique()

array(['ADEMIR DOS SANTOS', 'ANDREI GABRIEL MICHALAK',
       'BRUNO HORST DEMATHE', 'CARLOS STAHL', 'CLODOALDO DE SOUZA',
       'DANUBIA MOSER FRAGA', 'DEBORA CRISTINA HINDGES KLAUS',
       'EDERSON LUIS BRANCO', 'ENDRIU DE OLIVEIRA MELO',
       'FABIANA SCHRAMM FERREIRA', 'GABRIELA DE ANDRADE MONTEIRO',
       'JEAN CARLOS BRUCH', 'JEFFERSON CARDOSO DA SILVA EVANGELISTA',
       'JOAO ALMIR MORSCH', 'JOCIMAR KOBERNOVICZ',
       'JOSE MARIVALDO DA CONCEICAO SANTOS', 'JULIANE LUIZE DREWS',
       'LEANDRO NILSEN', 'LUIZ RICARDO PINHEIRO', 'MARCO AURELIO BARUFFI',
       'MAURICIO JOSE GORGES', 'PAULO SOARES DE OLIVEIRA',
       'RIVELINO STEIN', 'SONIA SARDAGNA NARLOCH',
       'VALFRIDO MAXIMILIANO MEYER SILVEIRA', 'VANDERLEI BERTOL',
       'VANESSA SCHMIDT', 'WILLIAN SILVA DE SOUSA'], dtype=object)

In [ ]:
'''
Criada uma lista apenas com os colaboradores necessários, 
pois a listagem original trás colaboradores de outras seções
'''

colaboradores = ['ADEMIR DOS SANTOS','CARLOS STAHL','DEBORA CRISTINA HINDGES KLAUS',
                 'EDERSON LUIS BRANCO','ENDRIU DE OLIVEIRA MELO','JEAN CARLOS BRUCH', 
                 'JEFFERSON CARDOSO DA SILVA EVANGELISTA','JOAO ALMIR MORSCH','JOCIMAR KOBERNOVICZ',
                 'JOSE MARIVALDO DA CONCEICAO SANTOS','LEANDRO NILSEN', 'LUIZ RICARDO PINHEIRO','MARCO AURELIO BARUFFI',
                 'MAURICIO JOSE GORGES','PAULO SOARES DE OLIVEIRA','RIVELINO STEIN', 'SONIA SARDAGNA NARLOCH',
                 'VALFRIDO MAXIMILIANO MEYER SILVEIRA','VANDERLEI BERTOL']

In [ ]:
'''
Realizando o filtro no dataframe para que
retorne apenas os colaboradores que fazem parte da seção
'''

def filtra_colaboradores():
  lista = []
  for i in dataframe['Colaborador']:
    if (i in colaboradores):
      lista.append(True)
    else:
      lista.append(False)
  return lista


selecao = filtra_colaboradores()
dataframe = dataframe[selecao]
dataframe.drop_duplicates(inplace=True)
dataframe.set_index('Conhecimento',inplace=True)
dataframe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Teste,Descrição_Norma,Colaborador,Rev_TLT,Rev_Norma
Conhecimento,,,,
31252,WPM-1453 PT : NÃO CONFORMIDADES E AÇÕES CORRET...,ADEMIR DOS SANTOS,11,18
31394,WPM-2094 PT : ADMINISTRAR GARANTIAS DE PRODUTO,ADEMIR DOS SANTOS,21,24
31742,WPM-2350 PT : PROCESSAR DEVOLUÇÕES DE CLIENTES,ADEMIR DOS SANTOS,6,11
30429,WPM-24063 PT : CONTROLE DE DOCUMENTOS DEPARTAM...,ADEMIR DOS SANTOS,2,4
31501,WPR-24891 PT : TRATATIVA PARA NÃO CONFORMIDADE...,ADEMIR DOS SANTOS,1,3
...,...,...,...,...
22455,WPR-7703 PT : PRAZOS DE RETENÇÃO PARA ARQUIVOS...,VANDERLEI BERTOL,4,5
25484,WPS-3942 PT : DETERMINACAO DA ESTABILIDADE ACE...,VANDERLEI BERTOL,5,6
25552,WPS-4069 PT : DETERMINAÇÃO DA COR GARDNER EM B...,VANDERLEI BERTOL,6,7


In [ ]:
'''
Exportando o dataframe para arquivo excel
'''
dataframe.to_excel("saida.xlsx")

In [ ]:
dataframe.to_csv('teste.csv')